<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [79]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [80]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [81]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [82]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [83]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [84]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [86]:
import requests

# SpaceX API endpoint voor alle lanceringen
spacex_url = "https://api.spacexdata.com/v4/launches/past"

# API-aanroep om de data op te halen
response = requests.get(spacex_url)

# Controleer of de API-aanroep succesvol is (statuscode 200 betekent succes)
print(f"API response status code: {response.status_code}")

# Haal de data op als JSON als de statuscode 200 is
if response.status_code == 200:
    data = response.json()
    print("Data successfully retrieved")
else:
    print("Failed to retrieve data")


API response status code: 200
Data successfully retrieved


Check the content of the response


In [87]:
# Print een voorbeeld van de opgehaalde data (de eerste paar lanceringen)
print(data[:2])  # De eerste twee records


[{'fairings': {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}, 'links': {'patch': {'small': 'https://images2.imgbox.com/94/f2/NN6Ph45r_o.png', 'large': 'https://images2.imgbox.com/5b/02/QcxHUb5V_o.png'}, 'reddit': {'campaign': None, 'launch': None, 'media': None, 'recovery': None}, 'flickr': {'small': [], 'original': []}, 'presskit': None, 'webcast': 'https://www.youtube.com/watch?v=0a_00nJ_Y88', 'youtube_id': '0a_00nJ_Y88', 'article': 'https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html', 'wikipedia': 'https://en.wikipedia.org/wiki/DemoSat'}, 'static_fire_date_utc': '2006-03-17T00:00:00.000Z', 'static_fire_date_unix': 1142553600, 'net': False, 'window': 0, 'rocket': '5e9d0d95eda69955f709d1eb', 'success': False, 'failures': [{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}], 'details': 'Engine failure at 33 seconds and loss of vehicle', 'crew': [], 'ships': [], 'capsules': [], 'payloads': ['5eb0e4b5b6c3bb0006eeb1e1

In [88]:
import pandas as pd

# Zet de data om naar een Pandas DataFrame
df = pd.DataFrame(data)

# Bekijk de eerste paar rijen van de DataFrame
print(df.head())


                                                                        fairings  \
0  {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}   
1  {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}   
2  {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}   
3  {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}   
4  {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [10]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [11]:
response.status_code

200

In [89]:
# Decodeer de API-response naar JSON
data = response.json()


In [90]:
from pandas import json_normalize

# Zet de JSON data om naar een Pandas DataFrame
df = json_normalize(data)

# Bekijk de eerste paar rijen van de DataFrame
print(df.head())


       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                                                                                            failures  \
0                                                [{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]   
1            [{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutd

In [91]:
# Bekijk de kolomnamen van de DataFrame
print(df.columns)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


Using the dataframe <code>data</code> print the first 5 rows


In [13]:
# Get the head of the dataframe
# Get the head of the data frame
df_head = df.head()
print(df_head)


       static_fire_date_utc  static_fire_date_unix    tbd    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False  False     0.0   
1                      None                    NaN  False  False     0.0   
2                      None                    NaN  False  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False  False     0.0   
4                      None                    NaN  False  False     0.0   

                     rocket  success  \
0  5e9d0d95eda69955f709d1eb    False   
1  5e9d0d95eda69955f709d1eb    False   
2  5e9d0d95eda69955f709d1eb    False   
3  5e9d0d95eda69955f709d1eb     True   
4  5e9d0d95eda69955f709d1eb     True   

                                                                                                                                                                                details  \
0                                                                                                                  

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [92]:
import requests

# Lijst om de raketversies op te slaan
rocket_versions = []

# Functie om raketversies op te halen met behulp van de rocket ID's
def get_rocket_info(df):
    for rocket_id in df['rocket']:
        # Maak een API-aanroep met het raket-ID
        response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}")
        
        # Haal de JSON-data op
        rocket_data = response.json()
        
        # Voeg de naam van de raket toe aan de lijst
        rocket_versions.append(rocket_data['name'])

# Voer de functie uit om raket-informatie op te halen
get_rocket_info(df)

# Voeg de opgehaalde raketversies toe aan de DataFrame
df['Rocket Version'] = rocket_versions

# Bekijk de eerste paar rijen van de bijgewerkte DataFrame
print(df[['rocket', 'Rocket Version']].head())


                     rocket Rocket Version
0  5e9d0d95eda69955f709d1eb       Falcon 1
1  5e9d0d95eda69955f709d1eb       Falcon 1
2  5e9d0d95eda69955f709d1eb       Falcon 1
3  5e9d0d95eda69955f709d1eb       Falcon 1
4  5e9d0d95eda69955f709d1eb       Falcon 1


* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [15]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [93]:
# Initialize an empty global list for BoosterVersion
BoosterVersion = []

# Check the initial state of BoosterVersion (it should be empty)
print(f"Initial BoosterVersion list: {BoosterVersion}")


Initial BoosterVersion list: []


In [94]:
# Function to get the booster version using the rocket IDs
def getBoosterVersion(data):
    global BoosterVersion
    for launch in data['rocket']:
        # Get rocket information using the SpaceX API and the rocket ID
        response = requests.get(f"https://api.spacexdata.com/v4/rockets/{launch}")
        
        # Check if the request was successful
        if response.status_code == 200:
            rocket_data = response.json()
            
            # Append the booster name (version) to the BoosterVersion list
            BoosterVersion.append(rocket_data.get('name', ''))
        else:
            # Handle any issues with the API request
            BoosterVersion.append('Unknown')


In [95]:
# Apply the getBoosterVersion function to populate the list
getBoosterVersion(df)

# Check the BoosterVersion list after applying the function
print(f"BoosterVersion list after applying getBoosterVersion: {BoosterVersion[:5]}")  # Show the first 5 items for brevity


BoosterVersion list after applying getBoosterVersion: ['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1']


In [96]:
# Apply the getBoosterVersion function to populate the list
getBoosterVersion(df)

# Check the BoosterVersion list after applying the function
print(f"BoosterVersion list after applying getBoosterVersion: {BoosterVersion[:5]}")  # Show the first 5 items for brevity


BoosterVersion list after applying getBoosterVersion: ['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1']


These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


# Function to get the booster version
def getBoosterVersion(data):
    for launch in data:
        rocket_id = launch.get('rocket')
        if rocket_id:
            response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}").json()
            BoosterVersion.append(response.get('name', ''))
Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


the list has now been update 


we can apply the rest of the  functions here:


In [98]:
def getPayloadData(data):
    for launch in data:
        payload_id = launch.get('payloads')
        if payload_id:
            # Aangezien dit een lijst is, halen we de eerste payload op
            response = requests.get(f"https://api.spacexdata.com/v4/payloads/{payload_id[0]}").json()
            PayloadMass.append(response.get('mass_kg', ''))  # Voeg de massa toe
            Orbit.append(response.get('orbit', ''))  # Voeg de baan (orbit) toe



In [99]:
def getLaunchSite(data):
    for launch in data:
        launchpad_id = launch.get('launchpad')
        if launchpad_id:
            response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{launchpad_id}").json()
            LaunchSite.append(response.get('name', ''))  # Voeg de naam van de lanceerplaats toe
            Longitude.append(response.get('longitude', ''))  # Voeg de longitude toe
            Latitude.append(response.get('latitude', ''))  # Voeg de latitude toe


In [100]:
def getCoreData(data):
    for launch in data:
        core_data = launch.get('cores')
        if core_data and core_data[0].get('core'):
            core_id = core_data[0].get('core')
            response = requests.get(f"https://api.spacexdata.com/v4/cores/{core_id}").json()
            Block.append(response.get('block', ''))
            ReusedCount.append(response.get('reuse_count', ''))
            Serial.append(response.get('serial', ''))
            Outcome.append(f"{core_data[0].get('landing_success', 'None')} + {core_data[0].get('landing_type', 'None')}")
            Flights.append(core_data[0].get('flight', ''))
            GridFins.append(core_data[0].get('gridfins', ''))
            Reused.append(core_data[0].get('reused', ''))
            Legs.append(core_data[0].get('legs', ''))
            LandingPad.append(core_data[0].get('landpad', ''))


In [101]:
import requests

# Lijst om de raketversies op te slaan
rocket_versions = []

# Functie om raketversies op te halen met behulp van de rocket ID's
def get_rocket_info(df):
    for rocket_id in df['rocket']:
        # Maak een API-aanroep met het raket-ID
        response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}")
        
        # Haal de JSON-data op
        rocket_data = response.json()
        
        # Voeg de naam van de raket toe aan de lijst
        rocket_versions.append(rocket_data['name'])

# Voer de functie uit om raket-informatie op te halen
get_rocket_info(df)

# Voeg de opgehaalde raketversies toe aan de DataFrame
df['Rocket Version'] = rocket_versions

# Bekijk de eerste paar rijen van de bijgewerkte DataFrame
print(df[['rocket', 'Rocket Version']].head())


                     rocket Rocket Version
0  5e9d0d95eda69955f709d1eb       Falcon 1
1  5e9d0d95eda69955f709d1eb       Falcon 1
2  5e9d0d95eda69955f709d1eb       Falcon 1
3  5e9d0d95eda69955f709d1eb       Falcon 1
4  5e9d0d95eda69955f709d1eb       Falcon 1


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
/tmp/ipykernel_68/3573834772.py in <module>
     40 
     41 # Create the DataFrame
---> 42 launch_df = pd.DataFrame(launch_dict)
     43 
     44 # Display the first few rows

~/conda/envs/python/lib/python3.7/site-packages/pandas/core/frame.py in __init__(self, data, index, columns, dtype, copy)
    612         elif isinstance(data, dict):
    613             # GH#38939 de facto copy defaults to False only in non-dict cases
--> 614             mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, typ=manager)
    615         elif isinstance(data, ma.MaskedArray):
    616             import numpy.ma.mrecords as mrecords

~/conda/envs/python/lib/python3.7/site-packages/pandas/core/internals/construction.py in dict_to_mgr(data, index, columns, dtype, typ, copy)
    463 
    464     return arrays_to_mgr(
--> 465         arrays, data_names, index, columns, dtype=dtype, typ=typ, consolidate=copy
    466     )
    467 

~/conda/envs/python/lib/python3.7/site-packages/pandas/core/internals/construction.py in arrays_to_mgr(arrays, arr_names, index, columns, dtype, verify_integrity, typ, consolidate)
    117         # figure out the index, if necessary
    118         if index is None:
--> 119             index = _extract_index(arrays)
    120         else:
    121             index = ensure_index(index)

~/conda/envs/python/lib/python3.7/site-packages/pandas/core/internals/construction.py in _extract_index(data)
    633             lengths = list(set(raw_lengths))
    634             if len(lengths) > 1:
--> 635                 raise ValueError("All arrays must be of the same length")
    636 
    637             if have_dicts:

ValueError: All arrays must be of the same lengthFinally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [103]:
import pandas as pd

# Maak een DataFrame van de verzamelde gegevens
df = pd.DataFrame({
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Longitude': Longitude,
    'Latitude': Latitude,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial
})

# Bekijk de eerste paar rijen van het DataFrame
print(df.head())



ValueError: All arrays must be of the same length

In [61]:
# Controleer het totale aantal rijen in de dataset
print(f"Total number of records: {len(df)}")

# Controleer het aantal Falcon 9 en Falcon 1 records
print(f"Falcon 9 records: {len(df[df['BoosterVersion'].str.contains('Falcon 9', case=False, na=False)])}")
print(f"Falcon 1 records: {len(df[df['BoosterVersion'].str.contains('Falcon 1', case=False, na=False)])}")


Total number of records: 107
Falcon 9 records: 99
Falcon 1 records: 5


In [62]:
# Controleer het aantal ontbrekende waarden in de kolom 'LandingPad'
missing_values = data_falcon9['LandingPad'].isnull().sum()
print(f"Number of missing values in LandingPad: {missing_values}")


Number of missing values in LandingPad: 0


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [32]:
import pandas as pd

# Voorbeeld van een woordenboek (launch_dict)
launch_dict = {
    'BoosterVersion': ['Falcon 9', 'Falcon Heavy', 'Falcon 9'],
    'PayloadMass': [5500, 15000, 12000],
    'Orbit': ['LEO', 'GTO', 'LEO'],
    'LaunchSite': ['CCAFS LC-40', 'KSC LC-39A', 'CCAFS LC-40'],
    'Longitude': [-80.577366, -80.603956, -80.577366],
    'Latitude': [28.561857, 28.573255, 28.561857],
    'Outcome': ['True + ASDS', 'False + None', 'True + ASDS'],
    'Flights': [1, 3, 2],
    'GridFins': [True, False, True],
    'Reused': [True, False, True],
    'Legs': [True, False, True],
    'LandingPad': ['LZ-1', 'None', 'LZ-1'],
    'Block': [5, 5, 5],
    'ReusedCount': [1, 0, 2],
    'Serial': ['B1049', 'B1052', 'B1048']
}

# Maak een Pandas DataFrame van het launch_dict
df = pd.DataFrame(launch_dict)

# Bekijk de eerste paar rijen van de DataFrame
print(df.head())


  BoosterVersion  PayloadMass Orbit   LaunchSite  Longitude   Latitude  \
0       Falcon 9         5500   LEO  CCAFS LC-40 -80.577366  28.561857   
1   Falcon Heavy        15000   GTO   KSC LC-39A -80.603956  28.573255   
2       Falcon 9        12000   LEO  CCAFS LC-40 -80.577366  28.561857   

        Outcome  Flights  GridFins  Reused   Legs LandingPad  Block  \
0   True + ASDS        1      True    True   True       LZ-1      5   
1  False + None        3     False   False  False       None      5   
2   True + ASDS        2      True    True   True       LZ-1      5   

   ReusedCount Serial  
0            1  B1049  
1            0  B1052  
2            2  B1048  


In [63]:
# Controleer het aantal ontbrekende waarden in de kolom 'LandingPad'
missing_values = data_falcon9['LandingPad'].isnull().sum()
print(f"Number of missing values in LandingPad: {missing_values}")



Number of missing values in LandingPad: 0


Show the summary of the dataframe


In [33]:
# Statistische samenvatting van de DataFrame (alleen numerieke kolommen)
print(df.describe())

# Informatie over de DataFrame, inclusief kolomnamen en datatypes
print(df.info())



        PayloadMass  Longitude   Latitude  Flights  Block  ReusedCount
count      3.000000   3.000000   3.000000      3.0    3.0          3.0
mean   10833.333333 -80.586229  28.565656      2.0    5.0          1.0
std     4856.267428   0.015352   0.006581      1.0    0.0          1.0
min     5500.000000 -80.603956  28.561857      1.0    5.0          0.0
25%     8750.000000 -80.590661  28.561857      1.5    5.0          0.5
50%    12000.000000 -80.577366  28.561857      2.0    5.0          1.0
75%    13500.000000 -80.577366  28.567556      2.5    5.0          1.5
max    15000.000000 -80.577366  28.573255      3.0    5.0          2.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   BoosterVersion  3 non-null      object 
 1   PayloadMass     3 non-null      int64  
 2   Orbit           3 non-null      object 
 3   LaunchSite      3 non-null    

In [102]:
# Check how many times each unique value appears in the 'BoosterVersion' column
print(df['BoosterVersion'].value_counts())


KeyError: 'BoosterVersion'

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [51]:
# Filter de data om alleen Falcon 9-lanceringen te behouden
data_falcon9 = df[df['BoosterVersion'].str.contains('Falcon 9', case=False, na=False)]

# Bekijk de eerste paar rijen van de gefilterde DataFrame
print(data_falcon9.head())


  BoosterVersion  PayloadMass Orbit   LaunchSite  Longitude   Latitude  \
0       Falcon 9         5500   LEO  CCAFS LC-40 -80.577366  28.561857   
2       Falcon 9        12000   LEO  CCAFS LC-40 -80.577366  28.561857   

       Outcome  Flights  GridFins  Reused  Legs LandingPad  Block  \
0  True + ASDS        1      True    True  True       LZ-1      5   
2  True + ASDS        2      True    True  True       LZ-1      5   

   ReusedCount Serial  
0            1  B1049  
2            2  B1048  


In [64]:
# Controleer het aantal ontbrekende waarden in de kolom 'LandingPad'
missing_values = data_falcon9['LandingPad'].isnull().sum()
print(f"Number of missing values in LandingPad: {missing_values}")


Number of missing values in LandingPad: 0


In [66]:
# Check how many times each unique value appears in the 'BoosterVersion' column
print(df['BoosterVersion'].value_counts())


Falcon 9        99
Falcon 1         5
Falcon Heavy     3
Name: BoosterVersion, dtype: int64


In [74]:
# Filter de data om alleen Falcon 9-lanceringen te behouden
data_falcon9 = df[df['BoosterVersion'].str.contains('Falcon 9', case=False, na=False)]

# Bekijk de eerste paar rijen van de gefilterde DataFrame
print(data_falcon9.head())

  BoosterVersion  PayloadMass Orbit    LaunchSite  Longitude   Latitude  \
5       Falcon 9          NaN   LEO  CCSFS SLC 40 -80.577366  28.561857   
6       Falcon 9          NaN   LEO  CCSFS SLC 40 -80.577366  28.561857   
7       Falcon 9        525.0   LEO  CCSFS SLC 40 -80.577366  28.561857   
8       Falcon 9        400.0   ISS  CCSFS SLC 40 -80.577366  28.561857   
9       Falcon 9        677.0   ISS  CCSFS SLC 40 -80.577366  28.561857   

       Outcome  Flights  GridFins  Reused   Legs LandingPad  Block  \
5  None + None        1     False   False  False       None    1.0   
6  None + None        1     False   False  False       None    1.0   
7  None + None        1     False   False  False       None    1.0   
8  None + None        1     False   False  False       None    1.0   
9  None + None        1     False   False  False       None    1.0   

   ReusedCount Serial  
5            0  B0003  
6            0  B0004  
7            0  B0005  
8            0  B0006  
9       

In [76]:
data_falcon9 = df[df['BoosterVersion'] == 'Falcon 9']


In [78]:
print(len(data_falcon9))

99


In [77]:
# Print het aantal Falcon 9-lanceringen
print(f"Aantal Falcon 9-lanceringen: {len(data_falcon9)}")


Aantal Falcon 9-lanceringen: 99


In [53]:
# Reset de index van de gefilterde DataFrame
data_falcon9.reset_index(drop=True, inplace=True)


In [71]:
# Print het aantal Falcon 9-lanceringen
print(f"Aantal Falcon 9-lanceringen: {len(data_falcon9)}")


Aantal Falcon 9-lanceringen: 99


Now that we have removed some values we should reset the FlgihtNumber column


In [41]:
# Reset de index van de gefilterde DataFrame, verwijder de oude index
data_falcon9.reset_index(drop=True, inplace=True)


In [72]:
data_falcon9 = df[df['BoosterVersion'] == 'Falcon 9']


In [73]:
print(len(data_falcon9))


99


In [44]:
# Controleer het aantal ontbrekende waarden in de kolom 'LandingPad'
missing_values = data_falcon9['LandingPad'].isnull().sum()
print(f"Number of missing values in LandingPad: {missing_values}")


Number of missing values in LandingPad: 0


In [45]:
print(df['BoosterVersion'].unique())


['Falcon 9' 'Falcon Heavy']


In [46]:
df['BoosterVersion'] = df['BoosterVersion'].str.strip()


In [47]:
data_falcon9 = df[df['BoosterVersion'].str.contains('Falcon 9')]


In [50]:
print(len(data_falcon9))

2


In [55]:
# Controleer of er een 'next' veld is in de API-response om meerdere pagina's op te halen
if 'next' in response.json():
    print("There is more data available on the next page.")


In [56]:
# Zorg ervoor dat de Falcon 9 filter correct wordt toegepast
data_falcon9 = df[df['BoosterVersion'].str.contains('Falcon 9', case=False, na=False)]

# Controleer het aantal Falcon 9-lanceringen
print(f"Aantal Falcon 9-lanceringen: {len(data_falcon9)}")


Aantal Falcon 9-lanceringen: 2


In [58]:
# Zorg ervoor dat de Falcon 9 filter correct wordt toegepast
data_falcon9 = df[df['BoosterVersion'].str.contains('Falcon 9', case=False, na=False)]

# Controleer het aantal Falcon 9-lanceringen
print(f"Aantal Falcon 9-lanceringen: {len(data_falcon9)}")


Aantal Falcon 9-lanceringen: 2


In [59]:
# Controleer het totale aantal rijen in de dataset
print(f"Total number of records: {len(df)}")

# Controleer het aantal Falcon 9 en Falcon 1 records
print(f"Falcon 9 records: {len(df[df['BoosterVersion'].str.contains('Falcon 9', case=False, na=False)])}")
print(f"Falcon 1 records: {len(df[df['BoosterVersion'].str.contains('Falcon 1', case=False, na=False)])}")


Total number of records: 3
Falcon 9 records: 2
Falcon 1 records: 0


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [37]:
data_falcon9.isnull().sum()

BoosterVersion    0
PayloadMass       0
Orbit             0
LaunchSite        0
Longitude         0
Latitude          0
Outcome           0
Flights           0
GridFins          0
Reused            0
Legs              0
LandingPad        0
Block             0
ReusedCount       0
Serial            0
FlightNumber      0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [39]:
import numpy as np

# Bereken het gemiddelde van de kolom PayloadMass
payload_mean = data_falcon9['PayloadMass'].mean()

# Print het berekende gemiddelde (optioneel)
print(f"Mean Payload Mass: {payload_mean}")

# Vervang de NaN-waarden in de kolom PayloadMass met het gemiddelde
data_falcon9['PayloadMass'].replace(np.nan, payload_mean, inplace=True)

# Controleer de kolom om te zien of de NaN-waarden zijn vervangen
print(f"Number of missing values in PayloadMass after replacement: {data_falcon9['PayloadMass'].isnull().sum()}")


Mean Payload Mass: 8750.0
Number of missing values in PayloadMass after replacement: 0


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


In [40]:
data_falcon9.to_csv('dataset_part_1.csv', index=False)


## Authors


data_falcon9.to_csv('dataset_part_1.csv', index=False)
<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright © 2021 IBM Corporation. All rights reserved.
